In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from torchvision import transforms
import cv2
from google.colab.patches import cv2_imshow
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
# Part 1: YOLO

def run_yolo(video_path, YOLO_model):
  cap = cv2.VideoCapture(video_path)
  writer = create_video_writer(cap)

  fps = int(cap.get(cv2.CAP_PROP_FPS))
  frame_interval = 3 * fps

  frame_count = 0
  sheet_music_lines = []
  bar_lines = []

  while True:

      success, img = cap.read()

      if not success:
          break

      if frame_count % frame_interval == 0:
        cropped_images, cropped_bars = predict_and_detect(YOLO_model, img, classes=[], conf=0.5)

        if len(cropped_images) > 1:  # There should only be 1 line of music per frame
              max_width = 0
              max_width_image = None

              for cropped_img in cropped_images:
                  height, width = cropped_img.shape[:2]
                  if width > max_width:
                      max_width = width
                      max_width_image = cropped_img

              cropped_images = [max_width_image]

        if len(cropped_images) != 0:
          sheet_music_lines.append(cropped_images[0])
          bars = []
          for bar in cropped_bars:
              bars.append(bar[1])

          bar_lines.append(bars)

      frame_count += 1

  writer.release()

  assert len(bar_lines) == len(sheet_music_lines)

  return sheet_music_lines

def remove_highlights(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    L, A, B = cv2.split(lab_img)

    threshold = 5 # Smaller threshold <-> less lenient
    mask = cv2.inRange(A, 128-threshold, 128+threshold) & cv2.inRange(B, 128 - threshold, 128 + threshold)

    # Invert mask to get non-gray areas
    mask_inv = cv2.bitwise_not(mask)
    gray_image_colored = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR)
    result = cv2.bitwise_and(img, img, mask=mask) + cv2.bitwise_and(gray_image_colored, gray_image_colored, mask=mask_inv)

    return result

def predict(chosen_model, img, classes=[], conf=0.5):
    if classes:
        results = chosen_model.predict(img, classes=classes, conf=conf)
    else:
        results = chosen_model.predict(img, conf=conf)

    return results


def predict_and_detect(chosen_model, img, classes=[], conf=0.5):
    results = predict(chosen_model, img, classes, conf=conf)
    cropped_images = []
    cropped_bars = []

    for result in results:
        for box in result.boxes:
            if int(box.cls[0]) == 0: # Class 2: line
              x1,x2,y1,y2 = int(box.xyxy[0][0]),int(box.xyxy[0][2]),int(box.xyxy[0][1]),int(box.xyxy[0][3])
              cropped_img = img[y1:y2, x1:x2]
              colors_removed_img = remove_highlights(cropped_img)
              cropped_images.append(colors_removed_img)

            if int(box.cls[0]) == 1: # Class 1: bar
              x1, x2 = int(box.xyxy[0][0]), int(box.xyxy[0][2])
              cropped_bars.append((x1, x2))

    # return img, results, cropped_images, cropped_bars
    return cropped_images, cropped_bars

# defining function for creating a writer (for mp4 videos)

def create_video_writer(video_cap, output_filename):
    # grab the width, height, and fps of the frames in the video stream.
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))

    # initialize the FourCC and a video writer object
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    writer = cv2.VideoWriter(output_filename, fourcc, fps,
                             (frame_width, frame_height))

    return writer

In [ ]:
# Part 2: Siamese

def run_siamese(sheet_music_lines, siamese_model):

  sheet_music_pairs = [
    (Image.fromarray(sheet_music_lines[i]), Image.fromarray(sheet_music_lines[i+1]))
    for i in range(len(sheet_music_lines) - 1)]

  transform = transforms.Compose([transforms.Resize((224, 224)),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                      std=[0.229, 0.224, 0.225])  # Same as ImageNet
                                  ])

  unique_lines = []
  unique_lines.append(sheet_music_pairs[0][0])

  for (img1, img2) in sheet_music_pairs:
    img_tensor_1 = transform(img1)
    img_tensor_2 = transform(img2)

    output1, output2 = siamese_model(img_tensor_1.unsqueeze(0), img_tensor_2.unsqueeze(0))

    distance = F.pairwise_distance(output1, output2).item()

    if distance >= 0.7:
      unique_lines.append(img2)

  return unique_lines


In [ ]:
# Part 3: ORB Sliding Window

def run_orb_sliding_window(sheet_music_lines):

  for image in sheet_music_lines:
    keypoints, descriptors = detect_and_compute(image)

  unique_music = []
  unique_music.append(sheet_music_lines[0])

  sheet_music_pairs = [
      (sheet_music_lines[i], sheet_music_lines[i+1])
      for i in range(len(sheet_music_lines) - 1)]

  bars_idx = 1

  for (img1, img2) in sheet_music_pairs:
    keypoints1, descriptors1 = detect_and_compute(np.array(img1))
    keypoints2, descriptors2 = detect_and_compute(np.array(img2))
    matches = match_descriptors(descriptors1, descriptors2, keypoints1, keypoints2, img1.shape, img2.shape)
    good_matches = matches[:int(len(matches) * 0.25)]

    homography_matrix = find_homography_and_check_overlap(img1, img2, keypoints1, keypoints2, good_matches) # Assign result to homography_matrix
    if homography_matrix is not None: # Check if homography_matrix is not None
      transformed_corners = get_transformed_corners(img1.shape, homography_matrix) # Use homography_matrix
      bbox = compute_bounding_box(transformed_corners)
      image_with_bbox = draw_bounding_box(img2, bbox)

      # Add cropped non-overlapping part of second image to array
      (x1, y1), (x2, y2) = bbox

      if abs(x2 - x1) < 0.8 * img2.shape[1]:
        unique_music.append(img2[y1:y2, x2:img2.shape[1]])

  stitch_and_save_images_to_pdf(unique_music=unique_music, pdf_file_path='output.pdf', align_left=True)

def detect_and_compute(image):
  orb = cv2.ORB_create(
      nfeatures=10000,
      scaleFactor=1.05,
      nlevels=6,
      edgeThreshold=0,
      firstLevel=0,
      WTA_K=4,
      scoreType=cv2.ORB_FAST_SCORE,
      patchSize=31,
      fastThreshold=10
  )
  keypoints, descriptors = orb.detectAndCompute(image, None)
  return keypoints, descriptors

def draw_keypoints(image, keypoints):
  image_with_keypoints = cv2.drawKeypoints(image, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
  cv2_imshow(image_with_keypoints)
  img2 = cv2.drawKeypoints(image, keypoints, None, color=(0,255,0), flags=0)
  cv2_imshow(img2)

def match_descriptors(descriptors1, descriptors2, keypoints1, keypoints2, img1_shape, img2_shape):
  matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
  initial_matches = matcher.match(descriptors1, descriptors2)

  biased_matches = []

  for initial_match in initial_matches:
    kp1 = keypoints1[initial_match.queryIdx].pt
    kp2 = keypoints2[initial_match.trainIdx].pt

    # keypoint indices: x: 0, y:1
    # img_shape indices: x: 1, y: 0
    kp1x, kp2x = kp1[0], kp2[0]
    kp2x = kp2

    x1_displacement = (img1_shape[1] - kp1[0]) / img1_shape[1]  # distance of keypoint 1 from the right end of image 1
    x2_displacement = kp2[0] / img2_shape[1]                    # distance of keypoint 2 from the left end of image 2

    # This is baseline distance between descriptors
    # For orb, maximum distance is 256 --> use for normalization
    descriptor_distance = initial_match.distance / 256

    initial_match.distance = x1_displacement + x2_displacement + 7 * descriptor_distance
    biased_matches.append(initial_match)

  biased_matches = sorted(biased_matches, key=lambda x: x.distance)
  return biased_matches

def draw_matches(image1, image2, keypoints1, keypoints2, matches):
  image_with_matches = cv2.drawMatches(image1, keypoints1, image2, keypoints2, matches, None)
  cv2_imshow(image_with_matches)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

def find_homography_and_check_overlap(image1, image2, keypoints1, keypoints2, matches):
    if len(matches) < 4:
        print("Not enough matches to compute homography.")
        return None # Return None instead of False

    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC)

    if H is not None:
        return H # Return the homography matrix
    else:
        return None # Return None instead of False

def get_transformed_corners(image_shape, H):
    h, w, = image_shape[:2]
    corners = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
    transformed_corners = cv2.perspectiveTransform(corners, H)
    return transformed_corners

def compute_bounding_box(corners):
    x_coords, y_coords = zip(*[pt[0] for pt in corners])
    x_max = int(max(x_coords))
    x_min = 0
    y_min = 0
    y_max = int(max(y_coords))
    return (x_min, y_min), (x_max, y_max)

def draw_bounding_box(image, bbox):
    (x_min, y_min), (x_max, y_max) = bbox
    output_image = image.copy()
    cv2.rectangle(output_image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    return output_image

def draw_axis_lines(image, x1, x2):
    output_image = image.copy()
    cv2.line(output_image, (x1, 10), (x2, 10), (0, 0, 255), 2)
    return output_image

def display_image_at_actual_size(image, save_path=None):
    """
    Display an image in its original size using plt.imshow(). Optionally save it as a PDF page.
    """
    h, w = image.shape[:2]
    dpi = plt.rcParams['figure.dpi']  # Use current DPI setting of matplotlib
    figsize = w / dpi, h / dpi  # Calculate the figure size in inches
    fig, ax = plt.subplots(figsize=figsize)
    ax.imshow(image)
    ax.axis('off')  # Turn off axis
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Remove padding around the image

    if save_path:
        fig.savefig(save_path, format='pdf')
    plt.close(fig)

def pad_image_to_height(image, target_height):
    """
    Pads the image vertically to match the target height.
    """
    h, w = image.shape[:2]
    if h == target_height:
        return image

    # Calculate padding
    pad_height = target_height - h
    pad_top = pad_height // 2
    pad_bottom = pad_height - pad_top

    # Pad the image using np.pad
    padded_image = np.pad(image, ((pad_top, pad_bottom), (0, 0), (0, 0)), mode='constant', constant_values=255)
    return padded_image

def pad_image_to_width(image, target_width, align_left=True):
    """
    Pads the image horizontally to match the target width.
    If `align_left` is True, pads only on the right side to align the image to the left.
    """
    h, w = image.shape[:2]
    pad_width = target_width - w

    if pad_width <= 0:
        return image  # No padding needed

    if align_left:
        # Padding only on the right side
        pad_left = 0
        pad_right = pad_width
    else:
        # Padding equally on both sides
        pad_left = pad_width // 2
        pad_right = pad_width - pad_left

    padded_image = np.pad(image, ((0, 0), (pad_left, pad_right), (0, 0)), mode='constant', constant_values=255)
    return padded_image

def crop_image_top_bottom(image, pixels=5):
    """Crop a specified number of pixels from the top and bottom of the image."""
    h, w = image.shape[:2]
    return image[pixels:h-pixels, :]  # Slice the image to remove `pixels` from top and bottom

def crop_image_width(image, crop_width):
    """Crop an image to a specified width."""
    return image[:, :crop_width]  # Keep only the first `crop_width` columns

def add_margin(image, margin):
    """
    Add a white margin around the image.
    """
    h, w = image.shape[:2]
    return np.pad(image, ((margin, margin), (margin, margin), (0, 0)), mode='constant', constant_values=255)

def stitch_and_save_images_to_pdf(unique_music, pdf_file_path, width_threshold=int(plt.rcParams['figure.dpi'] * 8.5), height_threshold=int(plt.rcParams['figure.dpi'] * 11), align_left=True, margin=10):
    """
    Stitch images together, and save each stitched result as a page in a PDF.
    """
    stitched_row = []
    stitched_rows = []  # Store all rows to stack vertically
    current_width = 0  # Track the current width of the stitched images
    current_height = 0  # Track the current height of the stitched rows
    max_row_height = 0  # Track the height of the tallest image in the row
    max_row_width = 0  # Track the width of the widest row

    pdf_pages = PdfPages(pdf_file_path)

    for line in unique_music:
        line_np = np.array(line)  # Convert to NumPy array if needed
        h, w = line_np.shape[:2]  # Get the height and width of the current image

        # If adding the next image will exceed the width threshold, handle the current row
        if current_width + w > width_threshold:
            remaining_width = width_threshold - current_width
            cropped_image = crop_image_width(line_np, remaining_width)
            stitched_row.append(cropped_image)

            # Pad all images in the row to match the height of the tallest image
            max_row_height = max(img.shape[0] for img in stitched_row)
            padded_row = [pad_image_to_height(img, max_row_height) for img in stitched_row]
            stitched_row_img = np.hstack(padded_row)
            stitched_row_img = crop_image_top_bottom(stitched_row_img)
            stitched_rows.append(stitched_row_img)

            # Update the total width of the stitched image (for padding rows)
            max_row_width = max(max_row_width, stitched_row_img.shape[1])
            current_height += max_row_height

            # Check if the current height exceeds the height threshold
            if current_height > height_threshold:
                # Pad all rows to the same width, aligning to the left if specified
                padded_rows = [pad_image_to_width(row, max_row_width, align_left=align_left) for row in stitched_rows]
                stitched_image = np.vstack(padded_rows)

                # Add margins around the entire stitched image
                stitched_image_with_margin = add_margin(stitched_image, 100)

                # Display and save the image with margins
                display_image_at_actual_size(stitched_image_with_margin, save_path=pdf_pages)

                # Reset for the next set of rows
                stitched_rows = []
                current_height = 0

            # Reset for the next row with the remaining portion of the image
            stitched_row = [line_np[:, remaining_width:]]  # Remaining part of the current image
            current_width = w - remaining_width
            max_row_height = h
        else:
            # Add the image to the current row
            stitched_row.append(line_np)
            current_width += w
            max_row_height = max(max_row_height, h)  # Update the tallest image in the row

    # Handle any remaining images in the last row
    if stitched_row:
        max_row_height = max(img.shape[0] for img in stitched_row)  # Ensure height consistency
        padded_row = [pad_image_to_height(img, max_row_height) for img in stitched_row]
        stitched_row_img = np.hstack(padded_row)
        stitched_rows.append(stitched_row_img)

    # Display and save any remaining stitched rows if they don't reach the height threshold
    if stitched_rows:
        # Pad all rows to the same width before stacking, aligning to the left if specified
        max_row_width = max(row.shape[1] for row in stitched_rows)  # Get max width again
        padded_rows = [pad_image_to_width(row, max_row_width, align_left=align_left) for row in stitched_rows]
        stitched_image = np.vstack(padded_rows)

        # Add margins around the entire stitched image
        stitched_image_with_margin = add_margin(stitched_image, margin)

        # Display and save the image with margins
        display_image_at_actual_size(stitched_image_with_margin, save_path=pdf_pages)

    pdf_pages.close()
